### Movies for U, V, W, T, S, Tracers - Aids to visualize the flow

In [2]:
#KRM

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

import matplotlib as mpl
#%matplotlib inline
from math import *

import scipy.io

import scipy as spy

from netCDF4 import Dataset

import pylab as pl

import os 

import sys

import seaborn as sns

In [3]:
lib_path = os.path.abspath('../../Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 


In [13]:
CanyonGrid='/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/gridGlob.nc'
CanyonGridOut = Dataset(CanyonGrid)

CanyonState='/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/stateGlob.nc'
CanyonStateOut = Dataset(CanyonState)

CanyonPtracers='/ocean/kramosmu/MITgcm/CanyonUpwelling/360x360x90_3Tr_LinProfiles_BarkleyHyd_GMREDI_FluxDiagnostics/run01/stateGlob.nc'
CanyonPtracersOut = Dataset(CanyonPtracers)



In [14]:
#for dimobj in CanyonStateOut.variables.values():
#    print dimobj

In [15]:
# General input

nx = 360
ny = 360
nz = 90
nt = 19 # t dimension size 

z = CanyonStateOut.variables['Z']
Time = CanyonStateOut.variables['T']

xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells
yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells

bathy = rout.getField(CanyonGrid, 'Depth')

hFacC = rout.getField(CanyonGrid, 'HFacC')
MaskC = rout.getMask(CanyonGrid, 'HFacC')

hFacW = rout.getField(CanyonGrid, 'HFacW') # (Z,Y,Xp1) for zonal fluxes
MaskW = rout.getMask(CanyonGrid,'HFacW') 

hFacS = rout.getField(CanyonGrid, 'HFacS') # (Z,Yp1,X) for meridional fluxes
MaskS = rout.getMask(CanyonGrid,'HFacS') 


In [16]:
# Load velocity components
uu = rout.getField(CanyonState,'U')
vv = rout.getField(CanyonState,'V')
W = rout.getField(CanyonState,'W')

U,V = rout.unstagger(uu,vv)



In [17]:
print(np.max(uu), np.min(uu))
print(np.max(vv), np.min(vv))
print(np.max(W), np.min(W))

Umax = np.max(uu)
Wmax = np.max(W)

(0.44712499, -0.14684488)
(0.13805558, -0.15556709)
(0.002690153, -0.0033017974)


In [18]:
import matplotlib.animation as animation

In [19]:
sns.set()
sns.set_style('white')
spdcmap = sns.cubehelix_palette(light=1, as_cmap=True)
divmap = sns.diverging_palette(255, 100, l=60, n=7, center="dark", as_cmap=True)

In [20]:
def Plot1(t,ax1):
    Uplot=np.ma.array(U[t,:,:,xslice],mask=MaskC[:,1:,xslice])
    csU = np.linspace(-1.0,1.0,num=25)
    mesh=ax1.contourf(yc[100:-1,200],z[:58],Uplot[:58,100:]/Umax,csU,cmap='RdYlGn_r')
    if t == 1:    
        cbar=plt.colorbar(mesh,ax=ax1,cmap ="RdYlGn_r")
    ax1.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax1.set_xlabel('Across-shore distance [m]')
    ax1.set_ylabel('Depth [m]')
    ax1.set_title('Alongshore velocity at day %0.1f' %(t/2.0))

def Plot2(t,ax2):    
    Uplot=np.ma.array(U[t,:,yslice,:],mask=MaskC[:,yslice,:])
    csU = np.linspace(-0.8,0.8,num=25)
    mesh=ax2.contourf(xc[200,100:260],z[:58],Uplot[:58,100:260]/Umax,csU,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax2)
    ax2.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax2.set_xlabel('Alongshore distance [m]')
    ax2.set_ylabel('Depth [m]')
    ax2.set_title('Alongshore velocity')
    
def Plot3(t,ax3):    
    uplane = np.ma.array(U[t,zslice2,:,1:],mask=MaskC[zslice2,1:,1:])
    vplane = np.ma.array(V[t,zslice2,1:,:],mask=MaskC[zslice2,1:,1:])
    spd = np.sqrt(uplane**2+vplane**2)
    csSpd = np.linspace(0,1,num=25)
    mesh=ax3.contourf(xc[150:250,1:],yc[150:250,1:],spd[150:250,:]/Umax,csSpd,cmap=spdcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax3)
    CS = ax3.contour(xc[150:250,1:],yc[150:250,1:],bathy[150:250,1:],[145],colors='k',linewidths=[2.0] )
    vect = ax3.quiver(xc[150:250:5, 1::15], yc[150:250:5, 1::15], uplane[150:250:5, ::15]/Umax, vplane[150:250:5, ::15]/Umax
                      , color='k')
    
    ax3.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax3.set_xlabel('Alongshore distance [m]')
    ax3.set_ylabel('Across-shore distance [m]')
    ax3.set_title('Horizontal velocity above shelf break')

def Plot4(t,ax4):
    Vplot=np.ma.array(V[t,:,:,xslice],mask=MaskC[:,:,xslice])
    csV = np.linspace(-0.2,0.2,num=25)
    mesh=ax4.contourf(yc[100:,200],z[:58],Vplot[:58,100:]/Umax,csV,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax4)
    ax4.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax4.set_xlabel('Across-shore distance [m]')
    ax4.set_ylabel('Depth [m]')
    ax4.set_title('Across-shore velocity')
    
def Plot5(t,ax5):    
    Vplot=np.ma.array(V[t,:,yslice,:],mask=MaskC[:,yslice,1:])
    csV = np.linspace(-0.3,0.3,num=25)
    mesh=ax5.contourf(xc[200,100:260],z[:58],Vplot[:58,100:260]/Umax,csV,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax5)
    ax5.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax5.set_xlabel('Alongshore distance [m]')
    ax5.set_ylabel('Depth [m]')
    ax5.set_title('Across-shore velocity')

def Plot6(t,ax6):
    uplane = np.ma.array(U[t,zslice,:,1:],mask=MaskC[zslice,1:,1:])
    vplane = np.ma.array(V[t,zslice,1:,:],mask=MaskC[zslice,1:,1:])
    spd = np.sqrt(uplane**2+vplane**2)
    csSpd = np.linspace(0,1,num=25)
    mesh=ax6.contourf(xc[150:250,1:],yc[150:250,1:],spd[150:250,:]/Umax,csSpd,cmap=spdcmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax6)
    vect = ax6.quiver(xc[150:250:5, 1::15], yc[150:250:5, 1::15], uplane[150:250:5, ::15]/Umax, vplane[150:250:5, ::15]/Umax,
            pivot='mid', color='k')
    CS = ax6.contour(xc[150:250,1:],yc[150:250,1:],bathy[150:250,1:],[148],colors='k',linewidths=[2.25] )
    ax6.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax6.set_xlabel('Alongshore distance [m]')
    ax6.set_ylabel('Across-shore distance [m]')
    ax6.set_title('Horizontal velocity at shelf break')

def Plot7(t,ax7):
    Wplot=np.ma.array(W[t,:,:,xslice],mask=MaskC[:,:,xslice])
    csW = np.linspace(-0.4,0.4,num=30)
    mesh=ax7.contourf(yc[100:,200],z[:58],Wplot[:58,100:]/Wmax,csW,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax7)
    ax7.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax7.set_xlabel('Across-shore distance [m]')
    ax7.set_ylabel('Depth [m]')
    ax7.set_title('Vertical velocity')

def Plot8(t,ax8):
    Wplot=np.ma.array(W[t,:,yslice,:],mask=MaskC[:,yslice,:])
    csW = np.linspace(-0.65,0.65,num=30)
    mesh=ax8.contourf(xc[200,100:260],z[:58],Wplot[:58,100:260]/Wmax,csW,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax8)
    ax8.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax8.set_xlabel('Alongshore distance [m]')
    ax8.set_ylabel('Depth [m]')
    ax8.set_title('Vertical velocity')
    
def Plot9(t,ax9):
    wplane = np.ma.array(W[t,zslice,:,:],mask=MaskC[zslice,:,:])
    csW = np.linspace(-1,1,num=30)
    mesh=ax9.contourf(xc[150:250,:],yc[150:250,:],wplane[150:250,:]/Wmax,csW,cmap='RdYlGn_r')
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax9)
    CS = ax9.contour(xc[150:250,1:],yc[150:250,1:],bathy[150:250,1:],[145],colors='k',linewidths=[2.25] )
    ax9.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax9.set_xlabel('Alongshore distance [m]')
    ax9.set_ylabel('Across-shore distance [m]')
    ax9.set_title('Vertical velocity at shelf break')

In [ ]:
## Animation
#N=5
xslice=180
yslice=200
zslice= 29 # shelf break index
zslice2= 23 

#Empty figures
fig,((ax1,ax2,ax3),(ax4, ax5,ax6),(ax7,ax8,ax9))  = plt.subplots(3, 3, figsize=(20, 14))

#Initial image
def init():
    Plot1(0,ax1)
    Plot2(0,ax2)
    Plot3(0,ax3)
    Plot4(0,ax4)
    Plot5(0,ax5)
    Plot6(0,ax6)
    Plot7(0,ax7)
    Plot8(0,ax8)
    Plot9(0,ax9)
    
    plt.tight_layout()
    #return[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]
    

def animate(tt):
    Plot1(tt,ax1)
    Plot2(tt,ax2)
    Plot3(tt,ax3)
    Plot4(tt,ax4)
    Plot5(tt,ax5)
    Plot6(tt,ax6)
    Plot7(tt,ax7)
    Plot8(tt,ax8)
    Plot9(tt,ax9)
    plt.tight_layout()
#The animation function (max frames=47)   


anim = animation.FuncAnimation(fig, animate, init_func=init,frames=18, interval = 200,blit=False, repeat=False)

##A line that makes it all work
mywriter = animation.FFMpegWriter()

##Save in current folder
anim.save('Vel_GMREDI_FluxDiagnostics_run01.mp4',writer=mywriter,fps=0.2, dpi = 400, bitrate = 1000000, codec = "libx264")

plt.show()

